In [36]:
%pip install matplotlib

     ---------------------------------------- 0.0/110.1 kB ? eta -:--:--
     ----------------------------------- -- 102.4/110.1 kB 3.0 MB/s eta 0:00:01
     -------------------------------------- 110.1/110.1 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
    --------------------------------------- 0.1/8.1 MB 3.6 MB/s eta 0:00:03
   - -------------------------------------- 0.3/8.1 MB 3.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.4/8.1 MB 3.4 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/8.1 MB 3.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/8.1 MB 3.2 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/8.1 MB 3.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.1 MB 3.3 MB/s eta 0:00:03
   ----- ---------------------------------- 1.2/8.1 MB 3.2 MB/s eta 0:00:03
   ------ --------------------------------- 1.3/8.1 MB 3.2 MB/s eta 0:00:03
   ------- ---


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from fuzzywuzzy import fuzz, process
import pandas as pd
import matplotlib.pyplot as plt
from unidecode import unidecode

In [3]:
df_event = pd.read_excel("ISL_Sample.xlsx")
# basis = set(df_shots['home_team'].unique().tolist()+df_shots['away_team'].unique().tolist())


In [4]:
df_shots = pd.read_excel("ISL_Sample_Shots.xlsx")
shot_teams = set(df_shots['home_team'].unique().tolist()+df_shots['away_team'].unique().tolist())

In [6]:
def best_fuzzy_match(team_name, choices, threshold=60):
    match, score = process.extractOne(team_name, choices, scorer=fuzz.token_sort_ratio)
    print(match, score)
    return match,score if score >= threshold else None

def normalize_text(txt):
    return unidecode(txt).lower()
# # Demo mapping home teams in teams1 to teams2 list
# for t1 in :
#     match = best_fuzzy_match(t1, shot_teams)
#     print(f"{t1} → {match}")


In [9]:
for i,row in df_event.iterrows() :
    print(i,row)
    break

0 home_team_full                              Mohun Bagan Super Giant
away_team_full                                         Bengaluru FC
home_score                                                        2
away_score                                                        1
datetime                                        2025-04-12 19:30:00
event_url         https://www.scoresway.com/en_GB/soccer/indian-...
Name: 0, dtype: object


In [17]:
df_shots.shape,df_event.shape

((163, 6), (155, 6))

In [7]:
import json
with open("./notebooks/Fetch games/team_alias.txt","r") as f :
    data = json.loads(f.read())

In [12]:
df_data = []
for name,alias in data.items():
    
    for al in alias :
        temp = {}
        temp['Input'] = al
        temp['Target'] = name
        temp['score'] = fuzz.ratio(name,al)
        temp['token_set_ratio'] = fuzz.token_set_ratio(name,al)
        temp['n_score'] = fuzz.ratio(normalize_text(name),normalize_text(al))
        temp['n_token_set_ratio'] = fuzz.token_set_ratio(normalize_text(name),normalize_text(al))
        # print(f"al : {al} | name : {name}")
        df_data.append(temp)


In [14]:
df = pd.DataFrame(df_data)

In [15]:
df.to_excel("Fuzzy Name Mapping.xlsx",index=False)

In [1]:
import os
import time
import shutil
from datetime import datetime, timedelta
from IPython.display import clear_output

# ANSI color codes
RED = "\033[91m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
RESET = "\033[0m"

def copy_files(src_dir, temp_dir):
    """Copy all files from source dir to temp dir (fresh snapshot)."""
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)  # remove old snapcshot
    os.makedirs(temp_dir, exist_ok=True)

    for filename in os.listdir(src_dir):
        src_file = os.path.join(src_dir, filename)
        dst_file = os.path.join(temp_dir, filename)
        if os.path.isfile(src_file):
            shutil.copy2(src_file, dst_file)

def tail_last_line(file_path):
    """Extract progress and error info from log file."""
    try:
        with open(file_path, 'r', errors="ignore") as f:
            valid = f.read().split("\n")
        valid_lines = [x.split("Working on Match")[-1].strip(" ").strip(".") 
                       for x in valid if "Working on Match" in x]
        error_lines = [x.split("INFO: Type     :")[-1].strip(" ") 
                       for x in valid if "INFO: Type     :" in x] + \
                      [x.split("Type         =>")[-1].strip(" ") 
                       for x in valid if "Type         =>" in x]
        if len(valid_lines) == 0 and len(error_lines) == 0:
            return None,None
            
        valid_last = valid_lines[-1] if valid_lines else "0/1"
        error_last = error_lines[-1] if error_lines else ""
        return valid_last, error_last
    except Exception:
        return "0/1", ""

def watch_directory(src_dir, interval=7):
    temp_dir = os.path.join(src_dir, "_temp_snapshot")
    last_progress = {}  # remember progress from last check

    try:
        while True:
            # Copy logs into temp dir
            copy_files(src_dir, temp_dir)

            rows = []
            stale_count=0
            
            for filename in os.listdir(temp_dir):
                filepath = os.path.join(temp_dir, filename)
                if os.path.isfile(filepath):
                    valid, err = tail_last_line(filepath)
                    if valid is None and err is None :
                        continue
                    try:
                        done, total = [int(x) for x in valid.split("/")]
                        percent = (done / total) * 100 if total > 0 else 0
                        progress_str = f"{percent:.2f}% ({valid})"
                    except Exception:
                        done, total, percent = 0, 1, 0
                        progress_str = "0.00% (0/1)"
                    
                    #remive stale logs
                    mtime = datetime.fromtimestamp(os.path.getmtime(filepath))
                    recent_threshold = datetime.now() - timedelta(hours=30)
                    is_recent = mtime >= recent_threshold
                    if not is_recent:
                        stale_count += 1
                        continue  # skip adding to rows
                    # detect progress change
                    prev_done = last_progress.get(filename, 0)
                    delta = done - prev_done
                    if delta > 0:
                        change_str = f"+{delta} 📈"
                    elif delta == 0:
                        change_str = "0 ➖"
                    else:
                        change_str = ""  # backward or reset
                    last_progress[filename] = done  # update snapshot

                    # check recent changes
                    mtime = datetime.fromtimestamp(os.path.getmtime(filepath))
                    recent = datetime.now() - timedelta(minutes=5)
                    is_recent = mtime >= recent
                    recent_flag = "✨" if is_recent else ""

                    if err:
                        message, emoji, color, group = err, f"❌{recent_flag}", RED, 2
                    elif done < total :
                        message, emoji, color, group = "Running", f"⏳{recent_flag}", YELLOW, 1
                    else:
                        message, emoji, color, group = "OK", f"✅{recent_flag}", GREEN, 3

                    rows.append((group, not is_recent, -percent, filename, message, progress_str, change_str, emoji, color))

            rows.sort()

            # print table
            running_count = sum(1 for r in rows if r[4] == "Running")
            error_count   = sum(1 for r in rows if r[4] != "Running" and r[4] != "OK")  # catches ❌
            ok_count      = sum(1 for r in rows if r[4] == "OK")

            # print table header
            clear_output(wait=True)
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n📅 Snapshot Time: {now} \n\nSeasons : | ⏳: {running_count} | ❌: {error_count} | ✅: {ok_count} | 💤: {stale_count}\n ")
            print(f"{'S.No.':<6} {'Filename':<65} {'Message':<25} {'Progress':<20} {'+/-':<10} {'Status':<10}")
            print("-" * 138)
            
            for idx, (_, _, _, filename, message, progress_str, change_str, emoji, color) in enumerate(rows, start=1):
                row = f"{len(rows) - idx + 1:<4} {' : '.join(filename.split('_-_')[:2]).replace('_',' '):<65} {message:<25} {progress_str:<20} {change_str:<13} {emoji:<10}"
                print(f"{color}{row}{RESET}")
                
            time.sleep(interval)
    except KeyboardInterrupt:
        print("\nStopped.")

if __name__ == "__main__":
    watch_directory(r"D:\runtime_logs", interval=7)



📅 Snapshot Time: 2025-08-23 19:27:09 

Seasons : | ⏳: 0 | ❌: 0 | ✅: 4 | 💤: 0
 
S.No.  Filename                                                          Message                   Progress             +/-        Status    
------------------------------------------------------------------------------------------------------------------------------------------
4    1. Lig : 2020 2021                                                OK                        100.00% (311/311)    0 ➖           ✅         
3    1. Lig : 2021 2022                                                OK                        100.00% (347/347)    0 ➖           ✅         
2    Liga I : 2017 2018                                                OK                        100.00% (268/268)    0 ➖           ✅         
1    Swiss Challenge League : 2022 2023                                OK                        100.00% (180/180)    0 ➖           ✅         

Stopped.
